This notebook relies on the following cmd line tools, ensure they are installed and in the system
- tippecanoe
- mapshaper
- aws cli

In [1]:
from pathlib import Path
import sys
import logging

from IPython.lib import backgroundjobs as bg

scripts_dir = Path('..').joinpath('src')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from pipelines.pipes import get_pipes, execution_order, filter_pipes

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("fiona").setLevel(logging.WARNING)
jobs = bg.BackgroundJobManager()

In [2]:
# def execute_pipes(pipes):
#     ordered_pipes = execution_order(pipes)
#     for pipe in ordered_pipes:
#         jobs.new(pipe().execute)

# # this code will execute all pipelines selected in background.

# mypipes_subset = filter_pipes(get_pipes().items(), ['eez_tiles'])
# execute_pipes(mypipes_subset)

In [3]:
get_pipes()

{'eez_intermediate': pipelines.intermediate_pipes.EEZIntermediatePipe.EEZIntermediatePipe,
 'mpaatlas_intermediate': pipelines.intermediate_pipes.MpaAtlasIntermediatePipe.MpaAtlasIntermediatePipe,
 'mpa_intermediate': pipelines.intermediate_pipes.MpasIntermediatePipe.MpasIntermediatePipe,
 'protectedseas_intermediate': pipelines.intermediate_pipes.ProtectedSeasIntermediatePipe.ProtectedSeasIntermediatePipe,
 'eez_tiles': pipelines.tiles_pipes.EEZTilesPipe.EEZTilesPipe,
 'mpaatlas_tiles': pipelines.tiles_pipes.MpaAtlasTilesPipe.MpaAtlasTilesPipe,
 'mpa_tiles': pipelines.tiles_pipes.MpasTilesPipe.MpasTilesPipe,
 'protectedseas_tiles': pipelines.tiles_pipes.ProtectedSeasTilesPipe.ProtectedSeasTilesPipe,
 'regions_tiles': pipelines.tiles_pipes.RegionsTilesPipe.RegionsTilesPipe,
 'habitats_precalc': pipelines.precalc_pipes.HabitatsPipe.HabitatsStatsPipe,
 'lfp_precalc': pipelines.precalc_pipes.LFPPipe.LFPStatsPipe,
 'eez_locations_precalc': pipelines.precalc_pipes.LocationsPipe.LocationsPip

In [4]:
mypipes_subset = filter_pipes(get_pipes(), ["mpaatlas_intermediate"])

In [5]:
# for n, pipe in mypipes_subset.items():
#     new_pipe = pipe()
#     new_pipe.extract().transform()

In [6]:
#!micromamba install pandera pandera-hypotheses pandera-io pandera-geopandas -c conda-forge -y

In [7]:
from pathlib import Path
from typing import Union
import pandas as pd
import geopandas as gpd
import numpy as np

from pipelines.utils import load_regions

from pipelines.output_schemas import (
    StablishmentStageSchema,
    FPLSchema,
    ProtectionLevelSchema,
    MPAsSchema,
    MPAsTableStatsSchema,
)

In [8]:
eez_folder = Path("../data/eez_intermediate").absolute()
mpa_folder = Path("../data/mpa_intermediate").absolute()
mpaatlas_folder = Path("../data/mpaatlas_intermediate").absolute()
protectedseas_folder = Path("../data/protectedseas_intermediate").absolute()

In [9]:
location_code = pd.read_csv(eez_folder.joinpath("locations_code.csv"))
mpa_intermediate = gpd.read_file(mpa_folder.joinpath("mpa_intermediate", "mpa_intermediate.shp"))

In [10]:
protectedseas_intermediate = gpd.read_file(
    protectedseas_folder.joinpath("protectedseas_intermediate", "protectedseas_intermediate.shp")
)

In [11]:
mpaatlas_intermediate = gpd.read_file(
    mpaatlas_folder.joinpath("mpaatlas_intermediate", "mpaatlas_intermediate.shp").as_posix()
)

In [46]:
def separate_parent_iso(df: pd.DataFrame, iso_column="location_i", separator=";") -> pd.DataFrame:
    df[iso_column] = (
        df[iso_column].str.replace(" ", "").str.replace(":", separator).str.split(separator)
    )
    return df.explode(iso_column)


def calculate_area(df: pd.DataFrame, output_area_column="area_km2") -> pd.DataFrame:
    df[output_area_column] = (df.to_crs("ESRI:54009")["geometry"].area / 10**6).round(2)
    return df


def calculate_global_area(
    df: pd.DataFrame, gby_col: list, output_area_column="area_km2", iso_column="location_i"
) -> pd.DataFrame:
    global_area = (
        df.groupby([*gby_col])
        .agg({output_area_column: "sum"})
        .reset_index()
        .assign(**{iso_column: "GLOB"})
    )
    return pd.concat([global_area, df], ignore_index=True)


def add_region_iso(df: pd.DataFrame, iso_column) -> pd.DataFrame:
    regions = load_regions()

    def find_region_iso(iso: str) -> Union[str, None]:
        filtered_regions = list(filter(lambda x: iso in x["country_iso_3s"], regions.get("data")))
        return filtered_regions[0]["region_iso"] if len(filtered_regions) > 0 else None

    return df.assign(region=lambda row: row[iso_column].apply(find_region_iso))


def mpaatlas_calculation(df: pd.DataFrame, gby_col: list) -> pd.DataFrame:
    regions = (
        df.groupby([*gby_col, "region"])
        .agg({"area_km2": "sum"})
        .reset_index()
        .rename(columns={"region": "location_i"})
    )

    return pd.concat(
        [
            regions,
            df.groupby([*gby_col, "location_i"]).agg({"area_km2": "sum"}).reset_index(),
        ],
        ignore_index=True,
    )


def protectedseas_calculation(df: pd.DataFrame, gby_col: list) -> pd.DataFrame:
    regions = (
        df.groupby([*gby_col, "region"])
        .agg({"area_km2": "sum"})
        .reset_index()
        .rename(columns={"region": "iso"})
    )
    return pd.concat(
        [
            regions,
            df.groupby([*gby_col, "iso"]).agg({"area_km2": "sum"}).reset_index(),
        ],
        ignore_index=True,
    )


def batch_export(df: pd.DataFrame, batch_size: int, schema: object, folder: Path, filename: str):
    prev = 0
    for idx, size in enumerate(range(batch_size, len(df.index) + batch_size, batch_size)):
        schema(df[(df.index > prev) & (df.index < size)]).to_csv(
            folder.joinpath(f"{filename}_{idx}.csv"),
            index=True,
            encoding="utf-8",
        )
        prev = size


def set_area(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(area_km2=df[["area_km2_y", "area_km2_x"]].max(axis=1))


def output(
    df: pd.DataFrame, iso_column: str, rep_d: dict, rename: dict, drop_cols: list
) -> pd.DataFrame:
    if iso_column:
        locations_code = pd.read_csv(
            eez_folder.joinpath("locations_code.csv"), keep_default_na=False
        )
        df = df.join(locations_code.set_index("code"), on=iso_column, how="left")
    return (
        df.replace(rep_d)
        .rename(columns=rename)
        .drop(columns=drop_cols)
        .assign(
            id=df.index + 1,
        )
        .set_index("id")
    )

In [13]:
mpa_atlas_table = (
    mpaatlas_intermediate.pipe(calculate_area)
)

In [14]:
test = (
    mpa_atlas_table.pipe(calculate_global_area, gby_col=["establishm"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(mpaatlas_calculation, gby_col=["establishm"])
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "establishm": {
                "actively managed": 4,
                "implemented": 6,
                "designated": 5,
                "proposed or committed": 3,
                "unknown": 1,
            }
        },
        rename={"establishm": "mpaa_establishment_stage", "area_km2": "area"},
        drop_cols=["location_i"],
    )
)

StablishmentStageSchema(test[~test.location.isna()].assign(year=2023, protection_status=1)).to_csv(
    mpaatlas_folder.joinpath("mpaatlas_stablishment.csv"), index=True
)

In [15]:
test2 = (
    mpa_atlas_table.pipe(calculate_global_area, gby_col=["protecti_1"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(mpaatlas_calculation, gby_col=["protecti_1"])
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "protecti_1": {
                "fully or highly protected": 1,
                "less protected or unknown": 2,
            }
        },
        rename={"protecti_1": "mpaa_protection_level", "area_km2": "area"},
        drop_cols=[],
    )
)
ProtectionLevelSchema(test2[~test2.location.isna()].assign(year=2023)).to_csv(
    mpaatlas_folder.joinpath("mpaatlas_protection_level.csv"), index=True
)

In [16]:
protected_seas_table = (
    protectedseas_intermediate.pipe(calculate_area)
)

In [17]:
test3 = (
    protected_seas_table.pipe(calculate_global_area, gby_col=["FPS_cat"], iso_column="iso")
    .pipe(separate_parent_iso, iso_column="iso")
    .replace(
        {
            "iso": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="iso")
    .pipe(protectedseas_calculation, gby_col=["FPS_cat"])
    .pipe(
        output,
        iso_column="iso",
        rep_d={
            "FPS_cat": {
                "highly": 1,
                "moderately": 2,
                "less": 3,
            }
        },
        rename={"FPS_cat": "fishing_protection_level", "area_km2": "area"},
        drop_cols=["iso"],
    )
)

FPLSchema(test3).to_csv(protectedseas_folder.joinpath("lfp.csv"), index=True)

In [18]:
test4 = mpa_intermediate.replace(
    {
        "PARENT_ISO": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
).pipe(
    output,
    iso_column="PARENT_ISO",
    rep_d={
        "STATUS": {
            "Adopted": 4,
            "implemented": 6,
            "Established": 6,
            "Designated": 5,
            "Proposed": 3,
            "Inscribed": 3,
            "unknown": 1,
        },
        "PA_DEF": {"0": 2, "1": 1},
        "STATUS_YR": {0: pd.NA},
    },
    rename={
        "PARENT_ISO": "iso",
        "PA_DEF": "protection_status",
        "REP_M_AREA": "area",
        "STATUS_YR": "year",
        "WDPA_PID": "wdpaid",
        "NAME": "name",
        "STATUS": "mpaa_establishment_stage",
    },
    drop_cols=["geometry", "WDPAID", "iso"],
).astype({"year": "Int64"})

prev = 0
for idx, size in enumerate(range(5000, len(test4.index) + 5000, 5000)):
    MPAsSchema(test4[(test4.index > prev) & (test4.index < size)]).to_csv(
        mpa_folder.joinpath(f"mpa_{idx}.csv"),
        index=True,
        encoding="utf-8",
    )
    prev = size

In [20]:
mpa_atlas_merge = mpa_atlas_table.pipe(separate_parent_iso, iso_column="location_i").replace(
    {
        "location_i": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
)

protectedseas_merge = protected_seas_table.pipe(separate_parent_iso, iso_column="iso").replace(
    {
        "iso": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
)

In [51]:
def filter_location(df: pd.DataFrame) -> pd.DataFrame:
    return df[~df.location.isna()]

In [68]:
Final = (
    test4.assign(mpa=test4.index)
    .merge(
        mpa_atlas_merge[["establishm", "wdpa_id", "protecti_1", "area_km2"]],
        left_on="wdpaid",
        right_on="wdpa_id",
        how="left",
    )
    .merge(
        protectedseas_merge[["site_id", "wdpa_id", "area_km2", "FPS_cat"]],
        left_on="wdpaid",
        right_on="wdpa_id",
        how="left",
    )
    .pipe(set_area)
    .pipe(filter_location)
    # .reset_index(drop=True)
    # .pipe(
    #     output,
    #     iso_column=None,
    #     rep_d={
    #         "protecti_1": {
    #             "fully or highly protected": 1,
    #             "less protected or unknown": 2,
    #         },
    #         "FPS_cat": {
    #             "highly": 1,
    #             "moderately": 2,
    #             "less": 3,
    #         },
    #         "establishm": {
    #             "actively managed": 4,
    #             "implemented": 6,
    #             "designated": 5,
    #             "proposed or committed": 3,
    #             "unknown": 1,
    #         },
    #     },
    #     rename={
    #         "establishm": "mpaa_establishment_stage",
    #         "protecti_1": "mpaa_protection_level",
    #         "FPS_cat": "fishing_protection_level",
    #     },
    #     drop_cols=[
    #         "wdpaid",
    #         "wdpa_id_x",
    #         "wdpa_id_y",
    #         "area_km2",
    #         "area_km2_x",
    #         "area_km2_y",
    #         "mpaa_establishment_stage",
    #         "protection_status",
    #         "name",
    #         "site_id",
    #     ],
    # )
)
# batch_export(Final, 5000, MPAsTableStatsSchema, mpa_folder, "mpa_join_mpatlas_prot")

In [82]:
protectedseas_merge.wdpa_id.unique().shape

(10786,)

In [84]:
protectedseas_merge.wdpa_id.str.replace(' ','').str.split(';').explode().unique().shape

(11984,)

In [79]:
test_Final = Final[~Final.area_km2.isna()][
    ["wdpaid", "name", "area_km2_x", "area", "area_km2_y", "FPS_cat", "area_km2", "protecti_1"]
]
test_Final["area_km2_1"] = test_Final.area_km2_y - test_Final.area_km2_x
test_Final[(test_Final.area_km2_1 != 0) & (test_Final.area_km2_1.notna())]

,wdpaid,name,area_km2_x,area,area_km2_y,FPS_cat,area_km2,protecti_1,area_km2_1
184,345888,Terres Australes Françaises,545381.47,0.000000,1646295.08,moderately,1646295.08,less protected or unknown,1100913.61
185,345888,Terres Australes Françaises,989240.07,0.000000,1646295.08,moderately,1646295.08,less protected or unknown,657055.01
186,345888,Terres Australes Françaises,120379.58,0.000000,1646295.08,moderately,1646295.08,fully or highly protected,1525915.50
398,354086_B,Macquarie Island,104492.63,104758.767956,104492.64,moderately,104492.64,less protected or unknown,0.01
1416,400012,Rose Atoll Marine National Monument,158.62,0.000000,35003.99,highly,35003.99,fully or highly protected,34845.37
...,...,...,...,...,...,...,...,...,...
15490,555636626,Refúgio De Vida Silvestre Do Arquipélago De Al...,669.41,674.724049,677.70,highly,677.70,fully or highly protected,8.29
15491,555636626,Refúgio De Vida Silvestre Do Arquipélago De Al...,6.31,674.724049,677.70,highly,677.70,fully or highly protected,671.39
16157,555655556,Big Bend Seagrasses Aquatic Preserve,2764.39,0.000000,3997.65,less,3997.65,less protected or unknown,1233.26
16160,555655559,Biscayne Bay Aquatic Preserve,279.83,0.000000,262.57,less,279.83,less protected or unknown,-17.26
